# keras conversion code

SNN 모델을 저장하되, IF 레이어는 ReLU로 바꿔서 저장하도록 한다. 대신 IF 레이어의 설정값은 따로 찍어서 보내기.

**주의점 : keras version 2.11이 들어간 환경에서 실행할 것! (KIST의 Neu+ 칩 구동환경과 일치시켜야 함)**

In [1]:
# path = '/data/hongwonseok/ECG-SNN/KIST/MLP/MLP_binary_NO_BatchNorm.pth'

path = '/home/hschoi/data/leehyunwon/ECG-SNN/SNN_MLP_ver5_LIF_poisson_binary_encoders187_hidden1000_encoderGradTrue_early30_lr0.001_threshold1.0_2024_10_10_15_50_39_lastEpoch.pt'
 
 

In [2]:
import torch

model = torch.load(path)
model

/tmp/ipykernel_1516302/3090495766.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path)


{'epoch': 149,
 'model_state_dict': OrderedDict([('hidden.0.weight',
               tensor([[ 0.0555, -0.1945, -0.2699,  ..., -1.0527, -1.1112, -0.7610],
                       [-0.6033, -0.1452,  0.2396,  ...,  0.0533,  0.0530,  0.2444],
                       [ 0.1006, -0.1668,  0.1413,  ...,  0.3818,  0.2243,  0.1764],
                       ...,
                       [-0.2028,  0.0975,  0.2866,  ...,  0.0589, -0.1674,  0.0449],
                       [-0.1642,  0.0153,  0.3809,  ..., -0.5676, -0.4606, -0.2609],
                       [-0.2396, -0.4548, -0.2539,  ...,  0.1766, -0.1016, -0.5567]],
                      device='cuda:0')),
              ('layer.0.weight',
               tensor([[ 1.2112, -2.4744,  0.1464,  ..., -0.8999, -1.1971,  0.3683],
                       [-1.1157,  1.9436, -0.1470,  ...,  1.1870,  1.0709, -0.3810]],
                      device='cuda:0'))]),
 'optimizer_state_dict': {'state': {0: {'step': tensor(51300.),
    'exp_avg': tensor([[ 6.7725e-05,  7.

In [3]:
weight = model['model_state_dict']
weight

OrderedDict([('hidden.0.weight',
              tensor([[ 0.0555, -0.1945, -0.2699,  ..., -1.0527, -1.1112, -0.7610],
                      [-0.6033, -0.1452,  0.2396,  ...,  0.0533,  0.0530,  0.2444],
                      [ 0.1006, -0.1668,  0.1413,  ...,  0.3818,  0.2243,  0.1764],
                      ...,
                      [-0.2028,  0.0975,  0.2866,  ...,  0.0589, -0.1674,  0.0449],
                      [-0.1642,  0.0153,  0.3809,  ..., -0.5676, -0.4606, -0.2609],
                      [-0.2396, -0.4548, -0.2539,  ...,  0.1766, -0.1016, -0.5567]],
                     device='cuda:0')),
             ('layer.0.weight',
              tensor([[ 1.2112, -2.4744,  0.1464,  ..., -0.8999, -1.1971,  0.3683],
                      [-1.1157,  1.9436, -0.1470,  ...,  1.1870,  1.0709, -0.3810]],
                     device='cuda:0'))])

In [4]:
weight.items()

odict_items([('hidden.0.weight', tensor([[ 0.0555, -0.1945, -0.2699,  ..., -1.0527, -1.1112, -0.7610],
        [-0.6033, -0.1452,  0.2396,  ...,  0.0533,  0.0530,  0.2444],
        [ 0.1006, -0.1668,  0.1413,  ...,  0.3818,  0.2243,  0.1764],
        ...,
        [-0.2028,  0.0975,  0.2866,  ...,  0.0589, -0.1674,  0.0449],
        [-0.1642,  0.0153,  0.3809,  ..., -0.5676, -0.4606, -0.2609],
        [-0.2396, -0.4548, -0.2539,  ...,  0.1766, -0.1016, -0.5567]],
       device='cuda:0')), ('layer.0.weight', tensor([[ 1.2112, -2.4744,  0.1464,  ..., -0.8999, -1.1971,  0.3683],
        [-1.1157,  1.9436, -0.1470,  ...,  1.1870,  1.0709, -0.3810]],
       device='cuda:0'))])

In [5]:
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout, InputLayer
from tensorflow.keras.layers import ZeroPadding1D, Conv1D, BatchNormalization, ReLU, AveragePooling1D, Flatten, Dense

2024-10-14 06:38:48.000630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


일단 GPT에게 변환 코드 예시를 받아놨으니 그걸로 진행해보자.

**KIST용으로 바꿀 땐 hidden layer 뉴런 수가 1000개 임에 주의!!!!**

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, ReLU

def create_keras_model_SNN(num_classes, num_encoders, hidden_size, hidden_size_2):
    # 입력 레이어 정의
    inputs = Input(shape=(num_encoders,), name='input')
    
    # 첫 번째 히든 레이어
    x = Dense(hidden_size, use_bias=False, name='hidden1')(inputs)
    x = ReLU(name='relu1')(x)
    
    # 히든레이어 하나만 써서 학습하는 모델로 변경
    # # 두 번째 히든 레이어
    # x = Dense(hidden_size_2, use_bias=False, name='hidden2')(x)
    # x = ReLU(name='relu2')(x)
    
    # 출력 레이어
    predictions = Dense(num_classes, use_bias=False, name='output')(x)
    predictions = ReLU(name='output_relu')(predictions)  # IFNode를 ReLU로 치환

    # 모델 생성
    model = Model(inputs=inputs, outputs=predictions)
    
    return model

# 모델 생성 예시
keras_model = create_keras_model_SNN(num_classes=2, num_encoders=187, hidden_size=1000, hidden_size_2=1000)

# 모델 구조 확인
keras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


2024-10-14 06:38:51.759524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 06:38:51.761451: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
# keras 모델의 초기 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[-0.06594887  0.02362754 -0.04320738 ...  0.01735181 -0.0195576
  -0.05716163]
 [-0.06967717  0.02906438  0.01611836 ...  0.06454928 -0.05455538
  -0.0589961 ]
 [ 0.04759073  0.02531201 -0.04507742 ...  0.00087014 -0.033693
   0.02122914]
 ...
 [-0.02675686 -0.00266955  0.01243763 ...  0.00312337 -0.03835884
  -0.00937458]
 [ 0.03140049  0.06365955  0.0353533  ... -0.04931051  0.06389149
  -0.06963926]
 [ 0.0035329  -0.02193116  0.05596375 ... -0.05815507 -0.02655634
  -0.02050129]]
Layer: relu1
Layer: output
[[ 0.05290559  0.00850834]
 [ 0.05321538  0.02485186]
 [-0.03884175 -0.06127483]
 ...
 [ 0.05455922  0.02856022]
 [ 0.00052556  0.03839688]
 [-0.03651633  0.04642548]]
Layer: output_relu


가중치 추출하고 케라스 모델에 대입하기

왜인진 모르겠지만 cuda 커널 이미지가 안맞는다 하다가 또 된다고 하다가 그런다. 일단 시도해서 될 때 계속 진행하도록 한다.

In [8]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
print(torch.version.cuda)  # CUDA 버전 확인
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인

!export CUDA_LAUNCH_BLOCKING=1

# PyTorch에서 가중치 추출 및 Keras에 적용
pt_weights = {k: v.cpu().numpy() for k, v in weight.items()}

# Keras 가중치 설정
# pytorch의 모델 가중치는 (출력, 입력) 순서로 저장되고 케라스는 그 반대이므로 Transpose 필요
keras_model.get_layer('hidden1').set_weights([pt_weights['hidden.0.weight'].T]) # bias 빼기!
# keras_model.get_layer('hidden2').set_weights([pt_weights['hidden2.0.weight'].T]) # 레이어 1개만 사용하는 경우 빼야 함!
keras_model.get_layer('output').set_weights([pt_weights['layer.0.weight'].T])

# 잘 들어갔나 확인?
keras_model.summary()

2.4.1
11.8
True
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 모델 저장
keras_model.save('poisson_1000_one_layer_LIF.h5')

In [10]:
# 모델의 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[ 0.05554117 -0.60329217  0.10063299 ... -0.20282644 -0.1642053
  -0.2395675 ]
 [-0.19448015 -0.1452338  -0.16681203 ...  0.09751602  0.01531125
  -0.45479232]
 [-0.26989135  0.23955078  0.14125073 ...  0.2866377   0.38093516
  -0.25388843]
 ...
 [-1.0526779   0.05328737  0.38182843 ...  0.05886592 -0.56763756
   0.17660348]
 [-1.1112064   0.05303211  0.22429852 ... -0.16740501 -0.4606028
  -0.10161023]
 [-0.7610172   0.24437033  0.17644587 ...  0.04491612 -0.26091087
  -0.5566925 ]]
Layer: relu1
Layer: output
[[ 1.2111725  -1.1156695 ]
 [-2.4743867   1.9436427 ]
 [ 0.1464121  -0.1469736 ]
 ...
 [-0.89987695  1.1870453 ]
 [-1.1970752   1.0709147 ]
 [ 0.36830744 -0.38104096]]
Layer: output_relu


In [11]:
from tensorflow.keras.models import load_model

# 저장된 모델 로드
loaded_model = load_model('/home/hschoi/leehyunwon/ECG-SNN/to_keras_conversion/poisson_1000_one_layer_LIF.h5')


In [12]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 로드된 모델의 가중치 확인
for layer in loaded_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)


Layer: input
Layer: hidden1
[[ 0.05554117 -0.60329217  0.10063299 ... -0.20282644 -0.1642053
  -0.2395675 ]
 [-0.19448015 -0.1452338  -0.16681203 ...  0.09751602  0.01531125
  -0.45479232]
 [-0.26989135  0.23955078  0.14125073 ...  0.2866377   0.38093516
  -0.25388843]
 ...
 [-1.0526779   0.05328737  0.38182843 ...  0.05886592 -0.56763756
   0.17660348]
 [-1.1112064   0.05303211  0.22429852 ... -0.16740501 -0.4606028
  -0.10161023]
 [-0.7610172   0.24437033  0.17644587 ...  0.04491612 -0.26091087
  -0.5566925 ]]
Layer: relu1
Layer: output
[[ 1.2111725  -1.1156695 ]
 [-2.4743867   1.9436427 ]
 [ 0.1464121  -0.1469736 ]
 ...
 [-0.89987695  1.1870453 ]
 [-1.1970752   1.0709147 ]
 [ 0.36830744 -0.38104096]]
Layer: output_relu
